# Regression Models to be tried

Table of Contents

## 1. Algorithm for trying out some regression models

## 2. Getting the Cleaned and Processed Data

## 3. Quick Recap of the Data

## 4. Linear Regression

## 5. Random Forest Regression

## 6. XGBoost

## 7. Additional Models